In [ ]:
# https://data.worldbank.org/indicator/NY.GDP.MKTP.KD
from shifter_pandas.worldbank import WorldbankDatasource

shifter_ds = WorldbankDatasource("data/API_NY.GDP.MKTP.KD_DS2_en_csv_v2_3630701.zip")

df_pib = shifter_ds.datasource(wikidata_name=True)
df_pib.rename(columns={"Value": "GDP", "CountryName": "Region"}, inplace=True)
df_pib

In [ ]:
df_pib = df_pib.query("WikidataName in ['world', 'Switzerland']")
df_pib

In [ ]:
import plotly.express as px

fig = px.line(df_pib, x="Year", y="GDP", width=800, height=800, color="WikidataName")
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

from shifter_pandas.bp import UNITS_ENERGY, BPDatasource

shifter_ds = BPDatasource("data/bp-stats-review-2021-all-data.xlsx")

In [ ]:
df = shifter_ds.datasource(
    regions_filter=["Total World", "Switzerland"],
    types_filter=[
        "Primary Energy Consumption",
        "CO2 From Flaring",
        "CO2 Excluding Flaring",
    ],
    wikidata_name=True,
)
df

In [ ]:
import plotly.express as px

df["TypeUnitRegion"] = df["TypeUnit"] + "-" + df["WikidataName"]
fig = px.line(df, x="Year", y="Value", color="TypeUnitRegion", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
df_co2 = (
    df.query("Type in ['CO2 From Flaring', 'CO2 Excluding Flaring']")
    .groupby(["Year", "WikidataName"])
    .sum()
    .reset_index()
)
df_co2.rename(columns={"Value": "CO2"}, inplace=True)
df_co2

In [ ]:
import plotly.express as px

fig = px.line(df_co2, x="Year", y="CO2", width=800, height=800, color="WikidataName")
fig.update_layout(
    yaxis_type="log",
    xaxis={"rangeslider": {"visible": True}},
)
fig.show()

In [ ]:
import pandas as pd

df_co2_pib = pd.merge(
    df_co2,
    df_pib,
    how="inner",
    on=["Year", "WikidataName"],
)
df_co2_pib
df_pib

In [ ]:
import plotly.express as px

fig = px.line(
    df_co2_pib,
    x="CO2",
    y="GDP",
    width=800,
    height=800,
    color="WikidataName",
    hover_data=["CO2", "GDP", "Year"],
    markers=True,
)
fig.show()

In [ ]:
df_e_pib = pd.merge(
    df.query('Type == "Primary Energy Consumption"'),
    df_pib,
    how="inner",
    on=["Year", "WikidataName"],
)
df_e_pib.rename(columns={"Value": "PrimaryEnergyConsumption"}, inplace=True)
df_e_pib

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_e_pib,
    x="PrimaryEnergyConsumption",
    y="GDP",
    width=800,
    height=800,
    trendline="lowess",
    color="WikidataName",
    hover_data=["PrimaryEnergyConsumption", "GDP", "Year"],
)
fig.show()

In [ ]:
df_co2_pib2 = df_co2_pib.assign(CO2_P_PIB=df_co2_pib.CO2 / df_co2_pib.GDP)
df_co2_pib2

In [ ]:
import plotly.express as px

fig = px.line(
    df_co2_pib2,
    x="Year",
    y="CO2_P_PIB",
    width=800,
    height=800,
    title="Efficacité",
    color="WikidataName",
)
fig.show()

In [ ]:
for col in ["Region", "CountryCode", "IndicatorName", "IndicatorCode"]:
    if col in df_co2_pib:
        del df_co2_pib[col]
df_co2_pib

In [ ]:
for col in ["Region", "CountryCode", "IndicatorName", "IndicatorCode"]:
    if col in df_co2_pib:
        del df_co2_pib[col]
df_co2_pib

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_e_pib.query("WikidataName == 'world'"),
    x="PrimaryEnergyConsumption",
    y="GDP",
    trendline="ols",
    hover_data=["PrimaryEnergyConsumption", "GDP", "Year"],
)
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_co2_pib.query("WikidataName == 'world'"),
    x="CO2",
    y="GDP",
    trendline="ols",
    hover_data=["CO2", "GDP", "Year"],
)
fig.show()

In [ ]:
# Save
import plotly

# Data
df_co2_pib.to_pickle("gdp_co2_world.px", protocol=4)

# HTML file
plotly.offline.plot(
    fig, filename="gdp_co2_world.html", auto_open=False, include_plotlyjs="cdn"
)